In [1]:
from bson.objectid import ObjectId
from bson.son import SON
from datetime import datetime
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from pymongo import MongoClient
import re
from sshtunnel import SSHTunnelForwarder

# Connect to MongoDB

## Connection to remote instance

In [2]:
"""
with open('../keys.json') as k:
    keys = json.loads(k.read())
    MONGO_HOST = keys["SSH"]["MONGO_HOST"] # i.e. 10.9.13.14
    MONGO_DB = keys["SSH"]["MONGO_DB"] # i.e. dm_project 
    MONGO_USER = keys["SSH"]["MONGO_USER"] # i.e. studente
    MONGO_PASS = keys["SSH"]["MONGO_PASS"] # i.e. la password della vm
server = SSHTunnelForwarder(
    MONGO_HOST,
    ssh_username=MONGO_USER,
    ssh_password=MONGO_PASS,
    remote_bind_address=('127.0.0.1', 27017)
)
server.start() # remember to stop

client = MongoClient('127.0.0.1', server.local_bind_port) # server.local_bind_port is assigned local port
db = client[MONGO_DB]
db.list_collection_names()"""

'\nwith open(\'../keys.json\') as k:\n    keys = json.loads(k.read())\n    MONGO_HOST = keys["SSH"]["MONGO_HOST"] # i.e. 10.9.13.14\n    MONGO_DB = keys["SSH"]["MONGO_DB"] # i.e. dm_project \n    MONGO_USER = keys["SSH"]["MONGO_USER"] # i.e. studente\n    MONGO_PASS = keys["SSH"]["MONGO_PASS"] # i.e. la password della vm\nserver = SSHTunnelForwarder(\n    MONGO_HOST,\n    ssh_username=MONGO_USER,\n    ssh_password=MONGO_PASS,\n    remote_bind_address=(\'127.0.0.1\', 27017)\n)\nserver.start() # remember to stop\n\nclient = MongoClient(\'127.0.0.1\', server.local_bind_port) # server.local_bind_port is assigned local port\ndb = client[MONGO_DB]\ndb.list_collection_names()'

## Local instance

In [3]:
client = MongoClient("localhost", 27017)
db = client["dm_project"]
db.list_collection_names()

['vgchartz', 'twitch', 'twitter']

#### Count of documents in each collection

Twitch:

In [4]:
db.twitch.count()

c:\python\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


8204

Twitter:

In [5]:
db.twitter.count()

c:\python\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


87412

#### Sizes of collections

Twitch:

In [6]:
print(db.command("collstats", 'twitch')['size']  / 1024 / 1024, 'MB')

4989.248718261719 MB


Twitter:

In [7]:
print(db.command("collstats", 'twitter')['size'] / 1024 / 1024, 'MB')

25.574621200561523 MB


# Queries on Twitch Collection

We can load the list of games we are monitoring, in case we want to restrict our queries to these games:

In [8]:
with open("../top_50_games_to_monitor.json", "r") as f:
    game_list = json.load(f)
    top_games = game_list["selected_top_50_games"]["name"]

top_games[:5]

['Grand Theft Auto V',
 'Call of Duty: Black Ops III',
 'Overwatch',
 'The Elder Scrolls V: Skyrim',
 'Hearthstone']

### Average daily views, channels and popularity for each game in the top_games list

The following query returns the average daily viewers, channels and popularity for each game in the list of monitored games

In [9]:
# MongoDB version >= 4

# pipeline = [{"$unwind": "$data"},
#             {"$match" : { "data.game_name" : { "$in" : top_games },  }},
#             { "$group": {
#                 "_id": {"name": "$data.game_name",
#                         "day": { "$dayOfMonth": {"$toDate" : "$timestamp" }},
#                         "month": { "$month": {"$toDate" : "$timestamp" }},
#                         "year": { "$year": {"$toDate" : "$timestamp" }} },
#                 "avgViewers": { "$avg": '$data.viewers'},
#                            "avgChannels": { "$avg": '$data.channels'},
#                            "avgPopularity": { "$avg": '$data.game_popularity'}
#             }},
#             {"$sort": {"_id": 1} }]

In [10]:
twitch_daily_avg_pipe = [{"$unwind": "$data"},
                        {"$match" : { "data.game_name" : { "$in" : top_games },  }},
                        { "$group": {
                            "_id": {"name": "$data.game_name",
                                    "norm_name": "$data.game_norm_name",
                                    "year" :  { "$substr" : ["$timestamp", 0, 4 ] }, 
                                    "month" : { "$substr" : ["$timestamp", 5, 2 ] },     
                                    "day" :   { "$substr" : ["$timestamp", 8, 2 ] } },
                            "avgViewers": { "$avg": '$data.viewers'},
                            "avgChannels": { "$avg": '$data.channels'},
                            "avgPopularity": { "$avg": '$data.game_popularity'}
                            }},
                        {"$sort": {"_id": 1} }]

In [11]:
twitch_daily_avg = list(db.twitch.aggregate(twitch_daily_avg_pipe))

In [12]:
twitch_daily_avg[:5]

[{'_id': {'name': 'Apex Legends',
   'norm_name': 'apex legends',
   'year': '2019',
   'month': '06',
   'day': '13'},
  'avgViewers': 17491.647058823528,
  'avgChannels': 1434.0761245674742,
  'avgPopularity': 17115.401384083045},
 {'_id': {'name': 'Apex Legends',
   'norm_name': 'apex legends',
   'year': '2019',
   'month': '06',
   'day': '14'},
  'avgViewers': 15606.502083333333,
  'avgChannels': 1634.5229166666666,
  'avgPopularity': 15777.739583333334},
 {'_id': {'name': 'Apex Legends',
   'norm_name': 'apex legends',
   'year': '2019',
   'month': '06',
   'day': '15'},
  'avgViewers': 15781.329166666666,
  'avgChannels': 1643.1083333333333,
  'avgPopularity': 15442.404166666667},
 {'_id': {'name': 'Apex Legends',
   'norm_name': 'apex legends',
   'year': '2019',
   'month': '06',
   'day': '16'},
  'avgViewers': 18600.82463465553,
  'avgChannels': 1656.6492693110647,
  'avgPopularity': 18946.350730688937},
 {'_id': {'name': 'Apex Legends',
   'norm_name': 'apex legends',
   

The following are the games initially chosen as top games, but that have not been collected again on twitch.

In [13]:
set(top_games) - set([elem['_id']['name'] for elem in twitch_daily_avg])

{'Clam Man'}

It is easier to visualize the result of the query as a dataframe:

In [14]:
twitch_stats_df = json_normalize(twitch_daily_avg)

In [15]:
twitch_stats_df = twitch_stats_df.rename(index=str,
                                         columns={"_id.name": "name", "_id.norm_name": "norm_name",
                                                  "_id.day": "day", "_id.month": "month","_id.year": "year"})
twitch_stats_df = twitch_stats_df[['name', "norm_name", 'day', 'month', 'year', 'avgViewers', 'avgPopularity', 'avgChannels']]

In [16]:
twitch_stats_df.head()

,name,norm_name,day,month,year,avgViewers,avgPopularity,avgChannels
0,Apex Legends,apex legends,13,06,2019,17491.647059,17115.401384,1434.076125
1,Apex Legends,apex legends,14,06,2019,15606.502083,15777.739583,1634.522917
2,Apex Legends,apex legends,15,06,2019,15781.329167,15442.404167,1643.108333
3,Apex Legends,apex legends,16,06,2019,18600.824635,18946.350731,1656.649269
4,Apex Legends,apex legends,17,06,2019,19137.743750,18741.535417,1555.712500


### Average daily views, channels and popularity for every game streamed on twitch

We can also query all games, not just those in the list:

In [17]:
all_twitch_daily_avg_pipe = [{"$unwind": "$data"},
                        { "$group": {
                            "_id": {"name": "$data.game_name",
                                    "year" :  { "$substr" : ["$timestamp", 0, 4 ] }, 
                                    "month" : { "$substr" : ["$timestamp", 5, 2 ] },     
                                    "day" :   { "$substr" : ["$timestamp", 8, 2 ] } },
                            "avgViewers": { "$avg": '$data.viewers'},
                            "avgChannels": { "$avg": '$data.channels'},
                            "avgPopularity": { "$avg": '$data.game_popularity'}
                            }},
                        {"$sort": {"_id": 1} }]

In [18]:
all_twitch_daily_avg = list(db.twitch.aggregate(all_twitch_daily_avg_pipe))

In [19]:
all_twitch_daily_avg[:5]

[{'_id': {'name': ' 8 To Glory - The Official Game of the PBR',
   'year': '2019',
   'month': '06',
   'day': '14'},
  'avgViewers': 1.0,
  'avgChannels': 1.0,
  'avgPopularity': 0.0},
 {'_id': {'name': ' 8 To Glory - The Official Game of the PBR',
   'year': '2019',
   'month': '06',
   'day': '22'},
  'avgViewers': 5.0,
  'avgChannels': 1.0,
  'avgPopularity': 0.0},
 {'_id': {'name': ' F1 2019', 'year': '2019', 'month': '06', 'day': '20'},
  'avgViewers': 22.821428571428573,
  'avgChannels': 1.0,
  'avgPopularity': 20.0},
 {'_id': {'name': ' F1 2019', 'year': '2019', 'month': '06', 'day': '21'},
  'avgViewers': 481.517571884984,
  'avgChannels': 4.696485623003195,
  'avgPopularity': 482.17891373801916},
 {'_id': {'name': ' F1 2019', 'year': '2019', 'month': '06', 'day': '22'},
  'avgViewers': 207.31914893617022,
  'avgChannels': 2.5379939209726445,
  'avgPopularity': 201.72036474164133}]

In [20]:
all_twitch_daily_df = json_normalize(all_twitch_daily_avg)

In [21]:
all_twitch_daily_df = all_twitch_daily_df.rename(index=str,
                                         columns={"_id.name": "name", "_id.day": "day",
                                                  "_id.month": "month","_id.year": "year"})
all_twitch_daily_df = all_twitch_daily_df[['name', 'day', 'month', 'year', 'avgViewers', 'avgPopularity', 'avgChannels']]

In [22]:
all_twitch_daily_df.head()

,name,day,month,year,avgViewers,avgPopularity,avgChannels
0,8 To Glory - The Official Game of the PBR,14,06,2019,1.000000,0.000000,1.000000
1,8 To Glory - The Official Game of the PBR,22,06,2019,5.000000,0.000000,1.000000
2,F1 2019,20,06,2019,22.821429,20.000000,1.000000
3,F1 2019,21,06,2019,481.517572,482.178914,4.696486
4,F1 2019,22,06,2019,207.319149,201.720365,2.537994


We add the column 'daily_top', which is the game's rank in the ordered list of daily most spectated games on average (with respect to every game streamed on Twitch).

In [23]:
all_twitch_daily_df['daily_top'] = all_twitch_daily_df.sort_values(['day', 'month', 'year', 'avgViewers'], 
                                                           ascending=[True, True, True, False]).groupby(['day', 'month', 'year']).cumcount()
all_twitch_daily_df['daily_top'] = all_twitch_daily_df['daily_top'] + 1

In [24]:
all_twitch_daily_df.sort_values(['day', 'month', 'year', 'avgViewers'], ascending=[True, True, True, False]).head()

,name,day,month,year,avgViewers,avgPopularity,avgChannels,daily_top
43610,League of Legends,13,06,2019,139492.283737,140292.190311,2966.896194,1
29744,Fortnite,13,06,2019,117869.685121,116048.532872,7186.041522,2
33266,Grand Theft Auto V,13,06,2019,97704.619377,95741.335640,1057.024221,3
40294,Just Chatting,13,06,2019,97585.868512,97702.740484,1351.307958,4
21674,Dota 2,13,06,2019,69030.173010,69218.449827,756.505190,5


In [25]:
all_twitch_daily_df[all_twitch_daily_df['name'] == 'Tetris']

,name,day,month,year,avgViewers,avgPopularity,avgChannels,daily_top
80738,Tetris,13,06,2019,49.739130,48.155797,6.847826,691
80739,Tetris,14,06,2019,35.179104,35.605544,7.695096,877
80740,Tetris,15,06,2019,31.952070,32.525054,6.893246,972
80741,Tetris,16,06,2019,51.653595,47.714597,6.472767,757
80742,Tetris,17,06,2019,59.634409,65.215054,5.600000,656
80743,Tetris,18,06,2019,54.096280,57.597374,6.601751,712
80744,Tetris,19,06,2019,66.323913,63.771739,5.093478,615
80745,Tetris,20,06,2019,29.947712,29.187364,6.485839,946
80746,Tetris,21,06,2019,53.063596,53.901316,6.050439,682
80747,Tetris,22,06,2019,44.438053,40.663717,5.473451,808


### Daily statistics for a single game

In [26]:
mario_daily_pipe = [{"$unwind": "$data"},
                    {"$match" : { "data.game_name" : "Super Mario Bros."  }},
                    { "$group": {
                        "_id": {"name": "$data.game_name",
                                "year" :  { "$substr" : ["$timestamp", 0, 4 ] }, 
                                "month" : { "$substr" : ["$timestamp", 5, 2 ] },     
                                "day" :   { "$substr" : ["$timestamp", 8, 2 ] } },
                        "avgViewers": { "$avg": '$data.viewers'},
                        "avgChannels": { "$avg": '$data.channels'},
                        "avgPopularity": { "$avg": '$data.game_popularity'}
                        }},
                    {"$sort": {"_id": 1} }]

In [27]:
mario_daily = list(db.twitch.aggregate(mario_daily_pipe))

In [28]:
mario_daily[:3]

[{'_id': {'name': 'Super Mario Bros.',
   'year': '2019',
   'month': '06',
   'day': '13'},
  'avgViewers': 51.2027027027027,
  'avgChannels': 3.4414414414414414,
  'avgPopularity': 44.531531531531535},
 {'_id': {'name': 'Super Mario Bros.',
   'year': '2019',
   'month': '06',
   'day': '14'},
  'avgViewers': 13.919014084507042,
  'avgChannels': 2.9119718309859155,
  'avgPopularity': 12.535211267605634},
 {'_id': {'name': 'Super Mario Bros.',
   'year': '2019',
   'month': '06',
   'day': '15'},
  'avgViewers': 74.43236714975845,
  'avgChannels': 3.7028985507246377,
  'avgPopularity': 72.28985507246377}]

### Every document regarding a single game

In [29]:
mario_all = list(db.twitch.find({'data.game_name': 'Super Mario Bros.'},
               {'timestamp':1, 'data.$': 1}))

In [30]:
mario_all[0:3]

[{'_id': ObjectId('5d0f46b1f0bd064ad83b132e'),
  'timestamp': '2019-06-13 09:32:34.348514',
  'data': [{'channels': 2,
    'game__id': 509508,
    'game_box_large': 'https://static-cdn.jtvnw.net/ttv-boxart/Super%20Mario%20Bros.-272x380.jpg',
    'game_giantbomb_id': 15544,
    'game_logo_large': 'https://static-cdn.jtvnw.net/ttv-logoart/Super%20Mario%20Bros.-240x144.jpg',
    'game_name': 'Super Mario Bros.',
    'game_norm_name': 'super mario bros.',
    'game_popularity': 1,
    'viewers': 5}]},
 {'_id': ObjectId('5d0f46b2f0bd064ad83b132f'),
  'timestamp': '2019-06-13 09:35:34.009864',
  'data': [{'channels': 2,
    'game__id': 509508,
    'game_box_large': 'https://static-cdn.jtvnw.net/ttv-boxart/Super%20Mario%20Bros.-272x380.jpg',
    'game_giantbomb_id': 15544,
    'game_logo_large': 'https://static-cdn.jtvnw.net/ttv-logoart/Super%20Mario%20Bros.-240x144.jpg',
    'game_name': 'Super Mario Bros.',
    'game_norm_name': 'super mario bros.',
    'game_popularity': 1,
    'viewers': 

### Query on a single game and day

We can compile a regex in order to find all documents regarding, for instance, 20/06/2019.

In [31]:
day = re.compile("^2019-06-20 \d+:\d+:\d+.\d+$")

In [32]:
mario_0620 = list(db.twitch.find({'data.game_name': 'Super Mario Bros.',
                     'timestamp':  {"$regex" : day  }},
                    {'timestamp':1, 'data.$': 1}))

In [33]:
mario_0620[:3]

[{'_id': ObjectId('5d0f4d10f0bd064ad83b1f8d'),
  'timestamp': '2019-06-20 00:02:34.009747',
  'data': [{'channels': 18,
    'game__id': 509508,
    'game_box_large': 'https://static-cdn.jtvnw.net/ttv-boxart/Super%20Mario%20Bros.-272x380.jpg',
    'game_giantbomb_id': 15544,
    'game_logo_large': 'https://static-cdn.jtvnw.net/ttv-logoart/Super%20Mario%20Bros.-240x144.jpg',
    'game_name': 'Super Mario Bros.',
    'game_norm_name': 'super mario bros.',
    'game_popularity': 416,
    'viewers': 366}]},
 {'_id': ObjectId('5d0f4d11f0bd064ad83b1f8e'),
  'timestamp': '2019-06-20 00:05:34.004106',
  'data': [{'channels': 19,
    'game__id': 509508,
    'game_box_large': 'https://static-cdn.jtvnw.net/ttv-boxart/Super%20Mario%20Bros.-272x380.jpg',
    'game_giantbomb_id': 15544,
    'game_logo_large': 'https://static-cdn.jtvnw.net/ttv-logoart/Super%20Mario%20Bros.-240x144.jpg',
    'game_name': 'Super Mario Bros.',
    'game_norm_name': 'super mario bros.',
    'game_popularity': 416,
    'vi

### Statistics for a single day

In [34]:
top_twitch_0620_pipe = [{"$unwind": "$data"},
                        {"$match" : {"$and":
                                     [{ "data.game_name" : { "$in" : top_games }  },
                                      {'timestamp':  {"$regex" : day}}] }},
                        { "$group": {"_id":
                                     {"name": "$data.game_name"},
                                     "avgViewers": { "$avg": '$data.viewers'},
                                     "avgChannels": { "$avg": '$data.channels'},
                                     "avgPopularity": { "$avg": '$data.game_popularity'}
                                     }},
                        {"$sort": {"_id": 1} }]

In [35]:
top_twitch_0620 = list(db.twitch.aggregate(top_twitch_0620_pipe))

In [36]:
top_twitch_0620[:5]

[{'_id': {'name': 'Apex Legends'},
  'avgViewers': 15973.658333333333,
  'avgChannels': 1600.80625,
  'avgPopularity': 16233.808333333332},
 {'_id': {'name': 'Auto Chess'},
  'avgViewers': 3473.3145833333333,
  'avgChannels': 102.01666666666667,
  'avgPopularity': 3548.9583333333335},
 {'_id': {'name': 'Call of Duty: Advanced Warfare'},
  'avgViewers': 8.268436578171091,
  'avgChannels': 3.017699115044248,
  'avgPopularity': 7.899705014749262},
 {'_id': {'name': 'Call of Duty: Black Ops'},
  'avgViewers': 81.78646934460888,
  'avgChannels': 12.930232558139535,
  'avgPopularity': 83.1416490486258},
 {'_id': {'name': 'Call of Duty: Black Ops 4'},
  'avgViewers': 10145.0125,
  'avgChannels': 1122.56875,
  'avgPopularity': 10217.6875}]

### Game with highest average viewers for each day

In [37]:
highest_avgView_per_day_pipe = [
    {"$unwind": "$data"},
    { "$group": {
                "_id": {"name": "$data.game_name",
                        "day" :   { "$substr" : ["$timestamp", 8, 2 ] },
                        "month" : { "$substr" : ["$timestamp", 5, 2 ] },         
                        "year" :  { "$substr" : ["$timestamp", 0, 4 ] }, },                        
                "avgViewers": { "$avg": '$data.viewers'}
            }},
    {"$sort": {"avgViewers": -1}},
    {"$group": {"_id": {
        "day": "$_id.day",
        "month": "$_id.month",
        "year": "$_id.year" },    
                "max_views": {"$first": "$avgViewers"},
                "game": {"$first": "$_id.name"}}},
    {"$project": {
            "_id": 0,
            "day": "$_id.day",
            "month": "$_id.month",
            "year": "$_id.year",
            "max_views": 1,
            "game": 1} },
    {"$sort": {"day": 1}}]

In [38]:
highest_avgView_per_day = list(db.twitch.aggregate(highest_avgView_per_day_pipe))

In [39]:
highest_avgView_per_day[:5]

[{'max_views': 139492.28373702423,
  'game': 'League of Legends',
  'day': '13',
  'month': '06',
  'year': '2019'},
 {'max_views': 138835.44166666668,
  'game': 'League of Legends',
  'day': '14',
  'month': '06',
  'year': '2019'},
 {'max_views': 179458.00625,
  'game': 'League of Legends',
  'day': '15',
  'month': '06',
  'year': '2019'},
 {'max_views': 165168.76617954072,
  'game': 'League of Legends',
  'day': '16',
  'month': '06',
  'year': '2019'},
 {'max_views': 131107.91666666666,
  'game': 'Fortnite',
  'day': '17',
  'month': '06',
  'year': '2019'}]

We could write a very similar query to find, for each day, the game with the highest average popularity or number of channels.

# Queries on Twitter

In [40]:
with open("../dataset/top_50_games_to_monitor.json", "r") as f:
    game_list = json.load(f)
    tweet_top_games = game_list["selected_top_50_games"]["norm_name"]

tweet_top_games[:5]

['clam man',
 'apex legends',
 'call of duty ghosts',
 'call of duty black ops iii',
 "playerunknown's battlegrounds"]

### Daily total retweets, likes, and tweet count for each game in the top_games list

In [41]:
tweet_daily_pipe = [{"$match" : { "query" : { "$in" : tweet_top_games},  }},
                    { "$group": {"_id": {"name": "$query",
                                         "day": { "$dayOfMonth": "$date" },
                                         "month": { "$month": "$date" },
                                         "year": { "$year": "$date" }},
                                 "tot_retweets": { "$sum": "$retweets" },
                                 "tot_likes": { "$sum" : "$likes"},
                                 "tweet_count" : { "$sum": 1 }
                                 }},
                    {"$sort": {"_id": 1} }]

In [42]:
tweet_daily = list(db.twitter.aggregate(tweet_daily_pipe))

In [43]:
tweet_daily[0:4]

[{'_id': {'name': 'apex legends', 'day': 13, 'month': 6, 'year': 2019},
  'tot_retweets': 54,
  'tot_likes': 349,
  'tweet_count': 96},
 {'_id': {'name': 'apex legends', 'day': 14, 'month': 6, 'year': 2019},
  'tot_retweets': 140,
  'tot_likes': 1022,
  'tweet_count': 200},
 {'_id': {'name': 'apex legends', 'day': 15, 'month': 6, 'year': 2019},
  'tot_retweets': 160,
  'tot_likes': 1154,
  'tweet_count': 200},
 {'_id': {'name': 'apex legends', 'day': 16, 'month': 6, 'year': 2019},
  'tot_retweets': 173,
  'tot_likes': 1491,
  'tweet_count': 169}]

In [44]:
tweet_daily_df = json_normalize(tweet_daily)
tweet_daily_df = tweet_daily_df.rename(index=str,
                                       columns={"_id.name": "name", "_id.day": "day","_id.month": "month","_id.year": "year"})
tweet_daily_df = tweet_daily_df[['name', 'day', 'month', 'year', 'tot_retweets', 'tot_likes', 'tweet_count']]
tweet_daily_df.head()

,name,day,month,year,tot_retweets,tot_likes,tweet_count
0,apex legends,13,6,2019,54,349,96
1,apex legends,14,6,2019,140,1022,200
2,apex legends,15,6,2019,160,1154,200
3,apex legends,16,6,2019,173,1491,169
4,apex legends,17,6,2019,112,692,16


In [45]:
tweet_daily_df[tweet_daily_df['name'] == 'tetris']

,name,day,month,year,tot_retweets,tot_likes,tweet_count
577,tetris,13,6,2019,28,282,200
578,tetris,14,6,2019,726,4310,76
579,tetris,15,6,2019,60,597,200
580,tetris,16,6,2019,143,903,200
581,tetris,17,6,2019,175,830,200
582,tetris,18,6,2019,360,1830,110
583,tetris,19,6,2019,317,1547,200
584,tetris,20,6,2019,764,11352,200
585,tetris,21,6,2019,188,1452,200
586,tetris,22,6,2019,38,305,29


### Game with the highest retweet count for each day:

In [46]:
pipeline_highest_retweet_per_day = [{
    "$group": {"_id": {"name": "$query",
                       "day": {"$dayOfMonth": "$date"},
                       "month": {"$month": "$date"},
                       "year": {"$year": "$date"} },
               "tot_retweets": { "$sum": "$retweets"}} },
    {"$sort": {"tot_retweets": -1} },
    {"$group": {"_id": {"day": "$_id.day",
                        "month": "$_id.month",
                        "year": "$_id.year"},
                "max_retweets": {"$first": "$tot_retweets"},
                "game": {"$first": "$_id.name"} } },
    {"$project": {"_id": 0,
                  "day": "$_id.day",
                  "month": "$_id.month",
                  "year": "$_id.year",
                  "max_retweets": 1,
                  "game": 1 } },
    {"$sort": { "day": 1}}]

In [47]:
highest_retweet_per_day = list(db.twitter.aggregate(pipeline_highest_retweet_per_day))
highest_retweet_per_day

[{'max_retweets': 419,
  'game': 'fortnite',
  'day': 13,
  'month': 6,
  'year': 2019},
 {'max_retweets': 1878,
  'game': 'fortnite',
  'day': 14,
  'month': 6,
  'year': 2019},
 {'max_retweets': 9617,
  'game': 'pokémon gold/silver',
  'day': 15,
  'month': 6,
  'year': 2019},
 {'max_retweets': 3365, 'game': 'slots', 'day': 16, 'month': 6, 'year': 2019},
 {'max_retweets': 6082,
  'game': 'overwatch',
  'day': 17,
  'month': 6,
  'year': 2019}]

### Game with the highest likes for each day:

In [48]:
pipeline_highest_likes_per_day = [{
    "$group": {"_id": {"name": "$query",
                       "day": {"$dayOfMonth": "$date"},
                       "month": {"$month": "$date"},
                       "year": {"$year": "$date"} },
               "tot_likes": { "$sum": "$likes"}} },
    {"$sort": {"tot_likes": -1} },
    {"$group": {"_id": {"day": "$_id.day",
                        "month": "$_id.month",
                        "year": "$_id.year"},
                "max_likes": {"$first": "$tot_likes"},
                "game": {"$first": "$_id.name"} } },
    {"$project": {"_id": 0,
                  "day": "$_id.day",
                  "month": "$_id.month",
                  "year": "$_id.year",
                  "max_likes": 1,
                  "game": 1 } },
    {"$sort": { "day": 1}}]

In [49]:
highest_likes_per_day = list(db.twitter.aggregate(pipeline_highest_likes_per_day))
highest_likes_per_day

[{'max_likes': 7344, 'game': 'fortnite', 'day': 13, 'month': 6, 'year': 2019},
 {'max_likes': 34285, 'game': 'fortnite', 'day': 14, 'month': 6, 'year': 2019},
 {'max_likes': 48678,
  'game': 'pokémon gold/silver',
  'day': 15,
  'month': 6,
  'year': 2019},
 {'max_likes': 33085, 'game': 'fortnite', 'day': 16, 'month': 6, 'year': 2019},
 {'max_likes': 35726, 'game': 'fortnite', 'day': 17, 'month': 6, 'year': 2019},
 {'max_likes': 53061, 'game': 'fortnite', 'day': 18, 'month': 6, 'year': 2019},
 {'max_likes': 15202, 'game': 'fortnite', 'day': 19, 'month': 6, 'year': 2019},
 {'max_likes': 30358, 'game': 'fortnite', 'day': 20, 'month': 6, 'year': 2019},
 {'max_likes': 22664, 'game': 'fortnite', 'day': 21, 'month': 6, 'year': 2019},
 {'max_likes': 43675,
  'game': 'minecraft',
  'day': 22,
  'month': 6,
  'year': 2019},
 {'max_likes': 25375,
  'game': 'minecraft',
  'day': 23,
  'month': 6,
  'year': 2019},
 {'max_likes': 39009, 'game': 'fortnite', 'day': 24, 'month': 6, 'year': 2019},
 {'m

### Which is the day with the highest likes? And which is the one with the lowest values?

In [50]:
pipeline_highest_likes = [{
    "$group": {"_id": {"day": {"$dayOfMonth": "$date"},
                       "month": {"$month": "$date"},
                       "year": {"$year": "$date"} },
               "tot_likes": { "$sum": "$likes"}} },
    {"$sort": {"tot_likes": -1} },
    {"$group": {"_id": {"day": "$_id.day",
                        "month": "$_id.month",
                        "year": "$_id.year"},
                "max_likes": {"$first": "$tot_likes"}}},
    {"$project": {"_id": 0,
                  "day": "$_id.day",
                  "month": "$_id.month",
                  "year": "$_id.year",
                  "max_likes": 1} },
    {"$sort": { "max_likes": -1}}]

In [51]:
highest_likes = list(db.twitter.aggregate(pipeline_highest_likes))
print('Day with highest likes: \n\t', highest_likes[0],
      '\n\nDay with lowest likes: \n\t', highest_likes[-1])

Day with highest likes: 
	 {'max_likes': 200997, 'day': 27, 'month': 6, 'year': 2019} 

Day with lowest likes: 
	 {'max_likes': 22493, 'day': 13, 'month': 6, 'year': 2019}


# Total "buzz"

We are interested in using both Twitch and Twitter data for determing which game has the highest "buzz", that is for us the weighted sum of the standardized values of the average views on Twitch and the total retweets and likes on Twitter.

In [52]:
pip = [{"$match" : { "query" : { "$in" : tweet_top_games},  }},
       {"$group": {"_id": {"name": "$query",
                          "day": {"$dayOfMonth": "$date"},
                          "month": {"$month": "$date"},
                          "year": {"$year": "$date"}},
        "total_retweets": {"$sum": "$retweets"},
        "total_likes": {"$sum": "$likes"}}},
       {"$project": {"_id": 0,
                     "norm_name": "$_id.name",
                     "day": "$_id.day",
                     "month": "$_id.month",
                     "year": "$_id.year",
                     "total_retweets": 1,
                     "total_likes": 1}},
       {"$sort": {"day": 1}}]

In [53]:
tweet_stats_query_results = list(db.twitter.aggregate(pip))
tweet_stats_df = pd.DataFrame.from_records(tweet_stats_query_results)
tweet_stats_df.head()

,day,month,norm_name,total_likes,total_retweets,year
0,13,6,pokémon gold/silver,16,1,2019
1,13,6,hearthstone,588,37,2019
2,13,6,grand theft auto san andreas,1,2,2019
3,13,6,fortnite,7344,419,2019
4,13,6,playerunknown's battlegrounds,20,4,2019


In [54]:
#server.stop()

In [55]:
twitch_stats_df.head()

,name,norm_name,day,month,year,avgViewers,avgPopularity,avgChannels
0,Apex Legends,apex legends,13,06,2019,17491.647059,17115.401384,1434.076125
1,Apex Legends,apex legends,14,06,2019,15606.502083,15777.739583,1634.522917
2,Apex Legends,apex legends,15,06,2019,15781.329167,15442.404167,1643.108333
3,Apex Legends,apex legends,16,06,2019,18600.824635,18946.350731,1656.649269
4,Apex Legends,apex legends,17,06,2019,19137.743750,18741.535417,1555.712500


**Let's join *tweet_stats_df* and *twitch_stats_df***

In [56]:
twitch_stats_df.day = twitch_stats_df.day.astype(int)
twitch_stats_df.month = twitch_stats_df.month.astype(int)
twitch_stats_df.year = twitch_stats_df.year.astype(int)

In [57]:
combined_stats = pd.merge(tweet_stats_df, twitch_stats_df, on = ['norm_name', 'day', 'month', 'year'], how="outer")

# replace some twitch and twitter missing values with 0: if the value is missing, the game was not streamed or tweeted
idx = ['total_retweets', 'total_likes','avgViewers', 'avgPopularity', 'avgChannels']
combined_stats[idx] = combined_stats[idx].fillna(0)

combined_stats.head()

,day,month,norm_name,total_likes,total_retweets,year,name,avgViewers,avgPopularity,avgChannels
0,13,6,pokémon gold/silver,16.0,1.0,2019,Pokémon Gold/Silver,8.733813,7.705036,1.942446
1,13,6,hearthstone,588.0,37.0,2019,Hearthstone,21380.671280,21350.159170,225.629758
2,13,6,grand theft auto san andreas,1.0,2.0,2019,Grand Theft Auto: San Andreas,455.910035,436.757785,28.186851
3,13,6,fortnite,7344.0,419.0,2019,Fortnite,117869.685121,116048.532872,7186.041522
4,13,6,playerunknown's battlegrounds,20.0,4.0,2019,PLAYERUNKNOWN'S BATTLEGROUNDS,40519.429066,40224.128028,1299.048443


### Computing total buzz

In [58]:
rtw_mu = np.mean(combined_stats['total_retweets'])
rtw_sd = np.std(combined_stats['total_retweets'])
combined_stats['std_tot_rtw'] = (combined_stats['total_retweets'] - rtw_mu) / rtw_sd

like_mu = np.mean(combined_stats['total_likes'])
like_sd = np.std(combined_stats['total_likes'])

combined_stats['std_tot_likes'] = (combined_stats['total_likes'] - like_mu) / like_sd

vw_mu = np.mean(combined_stats['avgViewers'])
vw_sd = np.std(combined_stats['avgViewers'])

combined_stats['std_avgViewers'] = (combined_stats['avgViewers'] - vw_mu) / vw_sd

Now that the features are standardized, we combine them in order to create the buzz feature. Since data from twitch and twitter should have the same weight, we create the new feature as a weighted sum.

In [59]:
weight_rtw = 0.25
weight_likes = 0.25
weight_twitch = 0.5

In [60]:
combined_stats['buzz'] = (weight_likes*combined_stats.std_tot_likes
                              + weight_rtw*combined_stats.std_tot_rtw
                              + weight_twitch*combined_stats.std_avgViewers)

In [61]:
combined_stats[['name','day', 'month', 'year', 'std_avgViewers', 'std_tot_likes', 'std_tot_rtw', 'buzz']].head()

,name,day,month,year,std_avgViewers,std_tot_likes,std_tot_rtw,buzz
0,Pokémon Gold/Silver,13,6,2019,-0.524156,-0.274792,-0.207752,-0.382714
1,Hearthstone,13,6,2019,0.177858,-0.190319,-0.179223,-0.003456
2,Grand Theft Auto: San Andreas,13,6,2019,-0.509468,-0.277008,-0.206959,-0.375726
3,Fortnite,13,6,2019,3.347282,0.807409,0.123497,1.906367
4,PLAYERUNKNOWN'S BATTLEGROUNDS,13,6,2019,0.806519,-0.274202,-0.205375,0.283365


## In which day a game obtained the highest "buzz"?

In [62]:
combined_stats.loc[combined_stats['buzz'].idxmax()]

day                       27
month                      6
norm_name          minecraft
total_likes            86161
total_retweets         18887
year                    2019
name               Minecraft
avgViewers           15866.8
avgPopularity        15895.6
avgChannels          1051.62
std_tot_rtw          14.7587
std_tot_likes        12.4471
std_avgViewers   -0.00325809
buzz                 6.79982
Name: 653, dtype: object

## In which day a game obtained the lowest "buzz"?

In [63]:
combined_stats.loc[combined_stats['buzz'].idxmin()]

day                          13
month                         6
norm_name         mario kart ds
total_likes                   0
total_retweets                0
year                       2019
name                        NaN
avgViewers                    0
avgPopularity                 0
avgChannels                   0
std_tot_rtw           -0.208544
std_tot_likes         -0.277155
std_avgViewers        -0.524443
buzz                  -0.383647
Name: 32, dtype: object

## Which is the game with the highest cumulative "buzz"?

In [64]:
cumulative_buzz = combined_stats.groupby("norm_name")['buzz'].sum().reset_index()
cumulative_buzz.loc[cumulative_buzz['buzz'].idxmax()]

norm_name    fortnite
buzz           47.477
Name: 17, dtype: object

## Which is the game with the lowest cumulative "buzz"?

In [65]:
cumulative_buzz.loc[cumulative_buzz['buzz'].idxmin()]

norm_name    call of duty modern warfare iii
buzz                                -5.74795
Name: 9, dtype: object

## Games sorted by cumulative buzz

In [66]:
cumulative_buzz.sort_values('buzz', ascending = False).reset_index()

,index,norm_name,buzz
0,17,fortnite,47.477005
1,24,league of legends,26.292073
2,22,just chatting,16.632956
3,28,minecraft,15.934295
4,20,grand theft auto v,14.820556
5,33,overwatch,9.512418
6,14,dota ii,9.080688
7,0,apex legends,7.331140
8,37,slots,5.835994
9,11,counter-strike global offensive,5.826284


## Integration with VGChartz

We integrate the list of monitored games, sorted by their buzz, with the data downloaded from VGChartz.

The missing values are due to data not available on VGChartz for some games or for some Twitch streams.

In [67]:
vgsalesn = pd.read_csv('../dataset/vgsales_n.csv')

In [68]:
platform = vgsalesn.groupby('Norm_name')['Platform'].apply(', '.join)
cr_score = vgsalesn.groupby('Norm_name')['Critic_Score'].mean()
sales = vgsalesn.groupby('Norm_name')['Global_Sales'].sum()
u_score  = vgsalesn.groupby('Norm_name')['User_Score'].mean()
pub = vgsalesn.groupby('Norm_name')['Publisher'].apply(', '.join)
dev = vgsalesn.groupby('Norm_name')['Developer'].apply(', '.join)

In [69]:
vgsalesn = pd.DataFrame({'Norm_name' : sales.index, 'Global_Sales': sales,
                   'Platforms': platform,  'Critic_Score' : cr_score, 'User_Score' : u_score,
                  'Publisher' : pub, 'Developer' : dev})

vgsalesn = vgsalesn.reset_index(drop = 1)

In [70]:
merged = cumulative_buzz.merge(vgsalesn, left_on = 'norm_name', right_on = 'Norm_name', how = 'outer')
merged = merged[merged.norm_name.isin(tweet_top_games)].sort_values('buzz', ascending = False)
merged = merged.reset_index(drop = True).drop('Norm_name', 1)
merged['Global_Sales'] = merged['Global_Sales'].replace(0, np.nan)

In [71]:
merged

,norm_name,buzz,Critic_Score,Developer,Global_Sales,Platforms,Publisher,User_Score
0,fortnite,47.477005,8.500000,"Epic Games , Epic Games , Epic Games , Epic...",2.37,"PS4, XOne, NS, PC","Gearbox Software , Gearbox Software , Warner...",NaN
1,league of legends,26.292073,8.000000,"Riot Games , Riot Games",NaN,"PC, OSX","THQ , Riot Games",NaN
2,just chatting,16.632956,NaN,NaN,NaN,NaN,NaN,NaN
3,minecraft,15.934295,8.160000,"Mojang AB , Mojang , Mojang , Mojang , Moj...",33.83,"PC, X360, PS4, PS3, XOne, PSV, NS, WiiU, 3DS, ...","Mojang , Microsoft Studios , Sony Computer E...",NaN
4,grand theft auto v,14.820556,9.366667,"Rockstar North , Rockstar North , Rockstar N...",65.62,"PS3, PS4, X360, PC, XOne","Rockstar Games , Rockstar Games , Rockstar G...",9.0
5,overwatch,9.512418,9.000000,"Blizzard Entertainment , Blizzard Entertainme...",8.18,"PS4, XOne, PC","Blizzard Entertainment , Blizzard Entertainme...",NaN
6,dota ii,9.080688,NaN,"Valve , Valve , Valve",NaN,"PC, OSX, Linux","Valve Corporation , Valve Corporation , Valv...",NaN
7,apex legends,7.331140,NaN,"Respawn Entertainment , Respawn Entertainment...",NaN,"XOne, PC, PS4","Electronic Arts , Electronic Arts , Electron...",NaN
8,slots,5.835994,NaN,Telegames,NaN,2600,Sears,NaN
9,counter-strike global offensive,5.826284,NaN,"Valve Corporation , Valve Corporation , Valv...",NaN,"PC, XBL, PSN, OSX","Valve , Valve Corporation , Valve Corporatio...",NaN
